<a href="https://colab.research.google.com/github/futabato/LT/blob/master/2020/chroma_key/Video_ChromaKey.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

各セルのコードは `Shift + Enter` で実行することができます．
上から3つ目の

In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode
from IPython.display import Image
import cv2
import numpy as np

In [ ]:
def take_photo(filename='photo.png', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/png', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

In [ ]:
# リンクをクリックしてドライブをマウントしてください（その方が楽そう）
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
cd drive/My\ Drive

/content/drive/My Drive


In [ ]:
def ExtractionColor(frame, back_image):
    back_image = cv2.resize(back_image, dsize=(800, 800))
    frame = cv2.resize(frame, dsize=(800, 800))
    
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    hsv_min = np.array([30, 64, 0])
    hsv_max = np.array([90, 255, 255])
    
    src_image_mask = cv2.inRange(hsv, hsv_min, hsv_max)
    object_image = cv2.bitwise_and(back_image, back_image, mask = src_image_mask)
    extraction = cv2.bitwise_not(src_image_mask)
    embedded = cv2.bitwise_and(frame, frame, mask = extraction)
    ChromaKey = cv2.bitwise_or(object_image, embedded)
    
    return ChromaKey



def main():
    video_capture = cv2.VideoCapture(0)

    # imread() の中に好きな画像を指定してください
    # pngでないと動作しないかも
    back_image = cv2.imread("メディアン5.png")

    video_capture.set(cv2.CAP_PROP_FPS, 30)
    
    while True:
        ret, frame = video_capture.read()
        
        cv2.imshow("ChromaKey", ExtractionColor(frame, back_image))
    
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

    video_capture.release()
    cv2.destroyAllWindows()

if __name__ == '__main__':
    main()

error: ignored